The very first step is to download and install the FETPAPI wheel (which depends on FESAPI) : pip install fetpapi

Once FETPAPI wheel installed, you need to first import FESAPI (required dependency) and then fetpapi in order to be able to call its API.

In [ ]:
import fesapi
import fetpapi

Let's now connect to an ETP server by defining some few initialization parameters such as the URL and the authorization

In [ ]:
import uuid

etp_server_url = "wss://.../"
authorization = "Bearer ..."
initialization_params = fetpapi.InitializationParameters(str(uuid.uuid4()), etp_server_url)

Some ETP servers such as the OSDU RDDMS one require some extra header fields. Let's add them

In [ ]:
additionalHeaderField = fetpapi.MapStringString()
additionalHeaderField["data-partition-id"] = "osdu"
initialization_params.setAdditionalHandshakeHeaderFields(additionalHeaderField)

We can now create the ETP session.

In [ ]:
client_session = fetpapi.createClientSession(initialization_params, authorization)

And we can finally run the session in parallel not to block the main thread

In [ ]:
def start_etp_server(client_session):
    client_session.run()

from threading import Thread
t = Thread(target=start_etp_server, args=(client_session,), daemon=True)
t.start()

Let's wait for the connection to be effective

In [ ]:
from time import sleep, perf_counter
start_time = perf_counter()
while client_session.isEtpSessionClosed() and perf_counter() - start_time < 5:
    sleep(0.25)	
if client_session.isEtpSessionClosed():
    print("The ETP session could not be established in 5 seconds.")
else:
    print("Now connected to ETP Server")

Let's create a dataspace with the OSDU ACL mandatory information

In [ ]:
dataspace = fetpapi.Dataspace()
dataspace.path = "demo/PutHorizon"
dataspace.uri = "eml:///dataspace('" + dataspace.path + "')"
dataspace.customData = fetpapi.MapStringDataValue()
dataValue = fetpapi.DataValue()
aos = fetpapi.ArrayOfString()
dataValue.item = fetpapi.DataValueitem_t()
tmp = fetpapi.StringVector()
tmp.push_back("data.default.viewers@osdu.example.com")
aos.values = tmp
dataValue.item.set_ArrayOfString(aos)
dataspace.customData["viewers"] = dataValue
tmp.clear()
tmp.push_back("data.default.owners@osdu.example.com")
aos.values = tmp
dataValue.item.set_ArrayOfString(aos)
dataspace.customData["owners"] = dataValue
tmp.clear()
tmp.push_back("osdu-public-usa-dataset")
aos.values = tmp
dataValue.item.set_ArrayOfString(aos)
dataspace.customData["legaltags"] = dataValue
tmp.clear()
tmp.push_back("US")
aos.values = tmp
dataValue.item.set_ArrayOfString(aos)
dataspace.customData["otherRelevantDataCountries"] = dataValue

Let's put the dataspace to the ETP store (optional if it already exists)

In [ ]:
dataspaces = fetpapi.MapStringDataspace()
dataspaces["0"] = dataspace
successKeys = client_session.putDataspaces(dataspaces)
if (len(successKeys) == 1):
    print("Dataspace has been put")
else:
    print("Error when putting dataspace. Already present?")

Let's create a RESQML horizon in order to put it to the ETP store

In [ ]:
# Create a DataObjectRepository in order to store Energistics dataobjects in-memory.
repo = fesapi.DataObjectRepository()
# Create an unknown local depth oriented 3D CRS in meters with no offset and no rotation
local_3d_crs = repo.createLocalDepth3dCrs("b2129512-b8f9-4721-8a70-1abac53ef406", "Default CRS",
                                          0.0, 0.0, 0.0, 0.0,
                                          fesapi.eml20__LengthUom_m, 5215,
                                          fesapi.eml20__LengthUom_m, "Unknown",
                                          False)
repo.setDefaultCrs(local_3d_crs)
repo.setHdfProxyFactory(fetpapi.FesapiHdfProxyFactory(client_session))
hdf_proxy = repo.createHdfProxy("f8160b8f-0517-4c55-ab6e-ed8bcdc87111", "Hdf Proxy",
                                ".", "fake.h5",
                                fesapi.DataObjectRepository.openingMode_OVERWRITE)
hdf_proxy.setUriSource(dataspace.uri)
repo.setDefaultHdfProxy(hdf_proxy)
# We can now create plenty of Energistics dataobjects. Let's try a Horizon 2d grid.
horizon_feature = repo.createHorizon("c0f12836-41f4-44a8-a3fd-95ac78f6232d", "My horizon feature")
horizon_interpretation = repo.createHorizonInterpretation(horizon_feature, "dc217b29-8ceb-4b77-bdcc-6bcfd9cd3baf", "My horizon interpretation")
horizon_grid_2d_representation = repo.createGrid2dRepresentation(horizon_interpretation, "7721fb3c-39ba-4d59-ba0b-f9451706a94c", "My horizon representation")

Let's write the DataArray of the horizon directly on the ETP store

In [ ]:
# Create first a transaction in order to be able to rollback if necessary and to easier resolve dependencies to put
dataspacesToLock = fetpapi.StringVector()
dataspacesToLock.append(dataspace.uri)
transactionFailure = client_session.startTransaction(dataspacesToLock)
if (transactionFailure):
    print (transactionFailure)
# And let's set a 2x3 geometry on this 2d grid where the Z values equal hundred times the node index.
# The first (aka I) axis of this 2d grid is aligned on X and the second (aka J) axis of this 2d grid is aligned on Y.
# The spacing on the first axis between two node is 25m and the spacing on the second axis between two node is 50m
resqml_points = fesapi.DoubleArray(6)
for i in range(6):
    resqml_points.setitem(i, i*100)
horizon_grid_2d_representation.setGeometryAsArray2dOfExplicitZ(resqml_points.cast(), 2, 3, hdf_proxy,
                                                                0.0, 0.0, 0.0,
                                                               1.0, 0.0, 0.0, 25.0,
                                                               0.0, 1.0, 0.0, 50.0)

At this time, the data array has been put on the ETP store, we still need to put the XML parts before to commit the transaction

In [ ]:
repo.setUriSource(dataspace.uri)
dataobjects = fetpapi.MapStringDataObject()
allUuids = repo.getUuids()
for i, uuid in enumerate(allUuids):
    dataobjects[str(i)] = fetpapi.buildEtpDataObjectFromEnergisticsObject(repo, uuid)
successKeys = client_session.putDataObjects(dataobjects)
#for successKey in successKeys:
#    print(successKey)
# Finally commit the transaction now that we have put everything we wanted
transactionFailure = client_session.commitTransaction()
if (transactionFailure):
    print (transactionFailure)

We can now close the session

In [ ]:
repo.clear()
client_session.close()